<a href="https://colab.research.google.com/github/waazai/fb_chat/blob/main/fb_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%capture
# This cell will take time
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install huggingface_hub
!pip install -q diffusers transformers accelerate peft

Found existing installation: unsloth 2025.1.5
Uninstalling unsloth-2025.1.5:
  Successfully uninstalled unsloth-2025.1.5
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-5qraeybo/unsloth_67e4c56f20aa4e6199bfb293d2a794cf
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-5qraeybo/unsloth_67e4c56f20aa4e6199bfb293d2a794cf
  Resolved https://github.com/unslothai/unsloth.git to commit f6174818d6f3461ff5b8457ddd514f350866d32e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.5-py3-none-any.whl size=174716 sha256=7770075ce2bd719406497e7cac77ee976e53974944cc8263e51006f86d965ffc
  Stored in directory: /tmp/pip-ephem-wheel-cache-dtzgczrs/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# download and load competition dataset

from google.colab import drive
drive.mount('/content/drive')

from datasets import load_from_disk
from datasets import Dataset
dataset = load_from_disk("drive/MyDrive/Colab Notebooks/fbChat_2024_cleaned.hf")

Mounted at /content/drive


In [ ]:
dataset['train']['text'][0]

'<<SYS>>Generate a chat conversation, avoid repeating content<</SYS>>\n### User: …… ### Tsu-yun Chen: 我們考線上 ### Tsu-yun Chen: 但是姑且是把study guide填滿了 ### User: 提醒我一下 ### User: 星期五下午6.30 ### User: 去参加一个基督教会的活动 ### Tsu-yun Chen: ? ### Tsu-yun Chen: 還不錯 ### Tsu-yun Chen: 堅持集中了45分鐘 ### Tsu-yun Chen: erai ### User: 完了 ### User: 我看了俩小时康熙 ### Tsu-yun Chen: 幹嘛的 ### User: 就是集会吧 ### User: 去蹭饭（ ### Tsu-yun Chen: 乎欸 ### Tsu-yun Chen: 奇怪的笑點被狠狠地戳下去了 ### User: 谁的 ### Tsu-yun Chen: 這系列好可愛 ### User: 戳到笑点了 ### Tsu-yun Chen: 我好想睡'

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 256 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
# Load model and wrap with LoRA

model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0.1,
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.1.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 100,
    num_train_epochs = 1,
    #max_steps = 1000,
    learning_rate = 2e-5,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.001,
    lr_scheduler_type = "cosine",
    seed = 3407,
    output_dir = "outputs",
    report_to = "none",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30,517 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 3,814
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
10,2.784000
20,2.749300
30,2.695400
40,2.595600
50,2.287100
60,2.162100
70,1.951100
80,2.192900
90,1.993900
100,1.996600


In [ ]:
# save to google drive

model.save_pretrained("drive/MyDrive/Colab Notebooks/fb_chat2") # replace with your disire path
tokenizer.save_pretrained("drive/MyDrive/Colab Notebooks/fb_chat2")

('drive/MyDrive/Colab Notebooks/fb_chat2/tokenizer_config.json',
 'drive/MyDrive/Colab Notebooks/fb_chat2/special_tokens_map.json',
 'drive/MyDrive/Colab Notebooks/fb_chat2/tokenizer.json')

Inference

In [ ]:
from transformers import TextGenerationPipeline


# create pipeli
class MyPipeline(TextGenerationPipeline):
    def postprocess(pipeline ,dict):
      # dict_keys(['generated_sequence', 'input_ids', 'prompt_text'])
      tmp = tokenizer.batch_decode([dict['generated_sequence'][0][0]], skip_special_tokens=True)

      # return only 1 message
      ans = tmp[0][len(dict['prompt_text']):]
      # chat = ans.strip().split('###')

      # res=[]
      # answerer = 'Tsu-yun Chen'
      # for msg in chat:
      #   msg = msg.split(':')
      #   if len(msg) == 2:
      #     if msg[0].strip() == answerer:
      #       res.append(msg[1].strip())
      #     elif len(res)>0:
      #       break

      # return '\n'.join(res)
      return ans

FastLanguageModel.for_inference(model)
pipe = MyPipeline(
    task="text-generation",
    model = model,
    tokenizer = tokenizer,
    batch_size = 4,
    eos_token_id = model.config.eos_token_id,
    max_new_tokens = 256,
)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor

In [ ]:
manual = '''<<SYS>>Generate a chat conversation, avoid repeating content<</SYS>>\n### User: 講點什麼'''
print('\n'.join(manual.split('###')))
print('\n<<以下是生成的回覆>>\n')
print(f'### Tsu-yun Chen: {pipe(manual)}')

<<SYS>>生成一段對話，且避免重複內容<</SYS>>

 User: 講點什麼

<<以下是生成的回覆>>

### Tsu-yun Chen:  ### Tsu-yun Chen: 講點什麼 ### Tsu-yun Chen: 講點什麼 ### Tsu-yun Chen: 講點什麼 ### User: 我有點想看你講點什麼 ### User: 我有點想看你講點什麼 ### Tsu-yun Chen: 講點什麼 ### Tsu-yun Chen: 講點什麼 ### User: 我有點想看你講點什麼 ### User: 我有點想看你講點什麼 ### Tsu-yun Chen: 講點什麼 ### Tsu-yun Chen: 講點什麼 ### User: 我有點想看你講點什麼 ### User: 我有點想看你講點什麼 ### User: 我有點想看你講點什麼 ### User: 我有點想看你講點什麼 ### User: 我有點想看你講點什麼 ### User: 我有點想看你講點什麼 ### Tsu-yun Chen: 講點什麼 ### Tsu-yun Chen: 講點什麼 ### Tsu-yun Chen: �
